In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
from functools import reduce
from pyspark.sql.functions import col, when, lit

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,None,pyspark,idle,,,✔


SparkSession available as 'spark'.


In [28]:
#col_name='provided_to'
col_name='vendors_provided'

In [54]:
df=spark.read.json("s3://<bucket>/uglyjson.json")
df.printSchema()
df.count()

root
 |-- btid: string (nullable = true)
 |-- id_key: string (nullable = true)
 |-- id_value: string (nullable = true)
 |-- mtime: long (nullable = true)
 |-- operation: string (nullable = true)
 |-- organization: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- type: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- provided_to: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- region: string (nullable = true)
 |-- vbtid: string (nullable = true)
 |-- vendors_provided: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- type: string (nullable = true)

19190

In [57]:
all_fields=[field.name for field in df.schema.fields]
complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])

In [10]:
return_type = ArrayType(MapType(StringType(), StringType()))
@udf(returnType=return_type)
def extract_udf(column_value):
    result=[]
    for i in range(len(column_value)):
       d={}
       d["id"]=column_value[i][0]
       d["type"]=column_value[i][1]
       result.append(d)
    if len(result)==0:
       d={}
       d["id"]=""
       d["type"]=""
       result.append(d)
    return result

In [29]:
df2=df.limit(5)
df3=df2.select(col_name)
df3.show(5,False)

+------------------+
|vendors_provided  |
+------------------+
|[[sUAMimn,vendor]]|
|[[sUAMimn,vendor]]|
|[[lo,vendor]]     |
|[[7sbtFLa,vendor]]|
|[[7sbtFLa,vendor]]|
+------------------+

In [30]:
df4=df3.withColumn(col_name,explode(extract_udf(col_name)))
df4.show(5,False)

+----------------------------------+
|vendors_provided                  |
+----------------------------------+
|Map(type -> vendor, id -> sUAMimn)|
|Map(type -> vendor, id -> sUAMimn)|
|Map(type -> vendor, id -> lo)     |
|Map(type -> vendor, id -> 7sbtFLa)|
|Map(type -> vendor, id -> 7sbtFLa)|
+----------------------------------+

In [33]:
expanded = [col(col_name).getItem(k).alias(col_name+'.'+k) for k in [ n.name for n in  fields[col_name].elementType]]

In [34]:
df4.select(col_name, *expanded).drop(col_name).show(5,False)

+-------------------+---------------------+
|vendors_provided.id|vendors_provided.type|
+-------------------+---------------------+
|sUAMimn            |vendor               |
|sUAMimn            |vendor               |
|lo                 |vendor               |
|7sbtFLa            |vendor               |
|7sbtFLa            |vendor               |
+-------------------+---------------------+

In [ ]:
col_names=['provided_to','vendors_provided']

In [69]:
df_new=df.limit(50)
col_name='provided_to'
df_new1=df_new.withColumn(col_name,explode(extract_udf(col_name)))
expanded = [col(col_name).getItem(k).alias(col_name+'.'+k) for k in [ n.name for n in  fields[col_name].elementType]]
df_new2=df_new1.select("*", *expanded).drop(col_name)
df_new2.printSchema()

root
 |-- btid: string (nullable = true)
 |-- id_key: string (nullable = true)
 |-- id_value: string (nullable = true)
 |-- mtime: long (nullable = true)
 |-- operation: string (nullable = true)
 |-- organization: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- type: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- region: string (nullable = true)
 |-- vbtid: string (nullable = true)
 |-- vendors_provided: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- provided_to.id: string (nullable = true)
 |-- provided_to.type: string (nullable = true)

In [70]:
col_name='vendors_provided'
df_new3=df_new2.withColumn(col_name,explode(extract_udf(col_name)))
expanded = [col(col_name).getItem(k).alias(col_name+'.'+k) for k in [ n.name for n in  fields[col_name].elementType]]
df_new3=df_new3.select("*", *expanded).drop(col_name)
df_new3.printSchema()

root
 |-- btid: string (nullable = true)
 |-- id_key: string (nullable = true)
 |-- id_value: string (nullable = true)
 |-- mtime: long (nullable = true)
 |-- operation: string (nullable = true)
 |-- organization: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- type: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- region: string (nullable = true)
 |-- vbtid: string (nullable = true)
 |-- provided_to.id: string (nullable = true)
 |-- provided_to.type: string (nullable = true)
 |-- vendors_provided.id: string (nullable = true)
 |-- vendors_provided.type: string (nullable = true)

In [81]:
col_name='organization'
expanded = [col(col_name+'.'+k).alias(col_name+'.'+k) for k in [ n.name for n in  fields[col_name]]]
print (expanded)
df_new4=df_new3.select("*", *expanded).drop(col_name)
df_new4.printSchema()

[Column<organization.id AS `organization.id`>, Column<organization.type AS `organization.type`>]
root
 |-- btid: string (nullable = true)
 |-- id_key: string (nullable = true)
 |-- id_value: string (nullable = true)
 |-- mtime: long (nullable = true)
 |-- operation: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- region: string (nullable = true)
 |-- vbtid: string (nullable = true)
 |-- provided_to.id: string (nullable = true)
 |-- provided_to.type: string (nullable = true)
 |-- vendors_provided.id: string (nullable = true)
 |-- vendors_provided.type: string (nullable = true)
 |-- organization.id: string (nullable = true)
 |-- organization.type: string (nullable = true)